# Philadelphia Real Estate Price Estimator - Predict Prices

This notebook is loads the machine learning model created by the "create-model" notebook and allows you to use that model to predict real-estate sales prices.

If you haven't run the create-model notebook yet:

[Click here to open the training notebook directly.](./create-model.ipynb)

If've already run the create-model notebook, click on the "Play" button above and then scroll down to the interactive form in order to run the model.

In [52]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import scipy
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from joblib import load
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, Layout
from IPython.display import display, Markdown


In [2]:
zip_codes = sorted(['19146', '19122', '19106', '19136', '19147', '19115', '19118', '19125',
 '19144', '19128', '19139', '19103', '19104', '19119', '19123', '19134',
 '19130', '19152', '19116', '19127', '19148', '19154', '19111', '19120',
 '19140', '19133', '19126', '19124', '19150', '19138', '19151', '19143',
 '19131', '19141', '19114', '19149', '19107', '19102', '00000', '19121',
 '19132', '19145', '19142', '19135', '19137', '19153', '19129', '19112'])
census_tracts = sorted(['13 ', '157', '10 ', '1  ', '348', '366', '356', '386', '387', '158', '344',
 '17 ', '240', '242', '11 ', '384', '93 ', '385', '12 ', '220', '86 ', '235',
 '367', '257', '253', '255', '176', '125', '18 ', '256', '92 ', '94 ', '347',
 '209', '237', '83 ', '388', '231', '365', '210', '163', '42 ', '238', '362',
 '306', '90 ', '96 ', '802', '358', '273', '27 ', '288', '286', '25 ', '16 ',
 '270', '301', '146', '340', '28 ', '359', '19 ', '302', '390', '261', '262',
 '249', '115', '78 ', '310', '105', '141', '142', '372', '72 ', '199', '282',
 '353', '352', '23 ', '338', '376', '192', '269', '40 ', '24 ', '191', '309',
 '283', '132', '107', '82 ', '165', '29 ', '73 ', '198', '15 ', '339', '143',
 '389', '114', '276', '174', '245', '272', '3  ', '2  ', '71 ', '360', '357',
 '258', '175', '244', '290', '195', '252', '95 ', '215', '84 ', '254', '9  ',
 '383', '65 ', '241', '6  ', '85 ', '214', '162', '246', '144', '7  ', '264',
 '311', '22 ', '110',   '-', '20 ', '342', '166', '87 ', '21 ', '293', '361',
 '219', '268', '74 ', '337', '203', '363', '138', '77 ', '137', '33 ', '377',
 '112', '111', '292', '101', '200', '308', '5  ', '140', '139', '263', '98 ',
 '30 ', '31 ', '190', '260', '32 ', '204', '373', '346', '274', '312', '318',
 '281', '167', '201', '14 ', '145', '39 ', '313', '133', '278', '173', '8  ',
 '153', '267', '147', '364', '280', '70 ', '314', '265', '300', '345', '335',
 '375', '336', '36 ', '277', '4  ', '148', '149', '259', '156', '294', '113',
 '135', '266', '119', '205', '134', '355', '64 ', '382', '351', '188', '38 ',
 '202', '178', '279', '161', '37 ', '152', '151', '298', '118', '317', '299',
 '41 ', '168', '319', '305', '271', '341', '100', '287', '321', '66 ', '160',
 '121', '117', '211', '216', '307', '179', '63 ', '334', '69 ', '169', '172',
 '380', '381', '120', '136', '164', '60 ', '67 ', '171', '180', '55 ', '378',
 '247', '62 ', '184', '183', '379', '333', '122', '61 ', '54 ', '177', '316',
 '315', '206', '332', '56 ', '289', '91 ', '243', '208', '800', '213', '236',
 '170', '109', '102', '103', '331', '809', '207', '239', '108', '88 ', '50 ',
 '197', '320', '106', '349', '291', '79 ', '80 ', '330', '329', '326', '104',
 '81 ', '325', '323', '284', '285', '217', '212', '248', '275', '801', '218',
 '131'])
condition = {
    'Not Applicable': '0',
    'Newer Construction': '1',
    'Rehabilitated - new': '2',
    'Above Average': '3',
    'Rehabilitated - older': '4',
    'Average': '5',
    'Below Average': '6',
    'Vacant': '7',
    'Sealed': '8',
    'Structurally Compromised': '9',
}
grade = {
    'Superior': 7,
    'Excellent': 6,
    'Very Good': 5,
    'Good': 4,
    'Normal': 3,
    'Fair': 2,
    'Poor': 1
}

In [4]:
model = load(filename="real-estate-random-forest-100.joblib")

In [5]:
transformer = load(filename='real-estate-transformer.joblib')

The census tract for a particular property can be found here:

https://geocoding.geo.census.gov/geocoder/geographies/address?form

In [55]:
style = {'description_width': '50%'}

display(Markdown('''
# Philadelphia Real Estate Price Predictor

Please adjust the controls below to set the house attributes.  The "Estimated Sale Price" will adjust automatically.

To find the Census Tract associated with an address, see https://geocoding.geo.census.gov/geocoder/geographies/address?form
'''))

@interact
def predict_sales_price(
        transfer_year=widgets.IntSlider(value=2023, min=2000, max=2023, description="Sale Year", style=style),
        zip_code=widgets.Dropdown(options=zip_codes, value='19128', description="Zip Code", style=style), 
        census_tract=widgets.Dropdown(options=census_tracts, value='216', description="Census Tract", style=style), 
        central_air=widgets.Dropdown(options={'No':0, 'Yes':1}, value=1, description="Central Air?", style=style), 
        depth=widgets.IntSlider(value=100, min=0, max=500, description="Depth", style=style),
        frontage=widgets.IntSlider(value=57, min=0, max=500, description="Frontage", style=style),
        garage_spaces=widgets.IntSlider(value=1, min=0, max=5, description="Garage Spaces", style=style),
        exterior_condition=widgets.Dropdown(
            options=condition, 
            value='1', 
            description="Exterior Condition",
            style=style
        ), 
        interior_condition=widgets.Dropdown(
            options=condition, 
            value='1', 
            description="Interior Condition",
            style=style
        ), 
        fireplaces=widgets.IntSlider(value=2, min=0, max=10, description="Fireplaces", style=style),
        number_of_bathrooms=widgets.IntSlider(value=5, min=0, max=25, description="Bathrooms", style=style),
        number_of_bedrooms=widgets.IntSlider(value=5, min=0, max=20, description="Bedrooms", style=style),
        number_stories=widgets.IntSlider(value=3, min=1, max=4, description="Stories", style=style),
        quality_grade=widgets.Dropdown(
            options=grade, 
            value=6, 
            description="Quality Grade",
            style=style
        ), 
        total_area=widgets.IntSlider(value=5700, min=1, max=50000, description="Lot area (sq feet)", style=style),
        total_livable_area = widgets.IntSlider(value=3000, min=1, max=20000, description="Livable area (sq feet)", style=style),
        age = widgets.IntSlider(value=0, min=0, max=300, description="Home Age", style=style),
        
):
    x = pd.DataFrame({
        'census_tract':census_tract,
        'transfer_year':transfer_year,
        'central_air':central_air,
        'depth':depth,
        'exterior_condition':exterior_condition,
        'fireplaces':fireplaces,
        'frontage':frontage,
        'garage_spaces':garage_spaces,
        'interior_condition':interior_condition,
        'number_of_bathrooms':number_of_bathrooms,
        'number_of_bedrooms':number_of_bedrooms,
        'number_stories':number_stories,
        'quality_grade':quality_grade,
        'total_area':total_area,
        'total_livable_area':total_livable_area,
        'age':age,
        'zip_code':zip_code,
        },
        index = [0]
    )
    xt = transformer.transform(x)
    return 'Estimated Sale Price:' + str(model.predict(xt)[0])




# Philadelphia Real Estate Price Predictor

Please adjust the controls below to set the house attributes.  The "Estimated Sale Price" will adjust automatically.

To find the Census Tract associated with an address, see https://geocoding.geo.census.gov/geocoder/geographies/address?form


interactive(children=(IntSlider(value=2023, description='Sale Year', max=2023, min=2000, style=SliderStyle(des…